In [ ]:
import pandas as pd

In [ ]:
#df = pd.read_csv('query2.csv')
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
# Load your dataset
df = pd.read_csv('/content/gdrive/MyDrive/DATASET12/UP BIHAR.csv')

# Display the first few rows of the dataset
df.head()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-12-07T17:08:36.226Z,30.3157,80.2382,35.00,4.1,mb,32.0,118.0,5.074,0.76,...,2024-12-07T21:52:53.065Z,"58 km NNW of Dhārchula, India",earthquake,10.58,1.943,0.109,25.0,reviewed,us,us
1,2024-10-14T16:47:47.648Z,27.3416,86.7989,10.00,4.0,mb,19.0,158.0,1.424,0.73,...,2024-11-04T21:51:13.040Z,"30 km NW of Bhojpur, Nepal",earthquake,10.52,1.894,0.138,14.0,reviewed,us,us
2,2024-09-14T18:21:42.274Z,29.9563,82.1893,13.48,4.4,mb,28.0,83.0,3.458,0.79,...,2024-11-21T22:58:57.040Z,"75 km N of Jumla, Nepal",earthquake,9.36,4.014,0.117,22.0,reviewed,us,us
3,2024-08-30T22:01:59.157Z,28.6347,84.3475,10.00,4.1,mb,19.0,165.0,1.169,0.44,...,2024-11-09T21:18:50.040Z,"44 km N of Besisahar, Nepal",earthquake,11.30,1.920,0.119,19.0,reviewed,us,us
4,2024-08-03T16:28:40.558Z,29.9216,87.0502,10.00,4.3,mb,53.0,82.0,2.623,0.58,...,2024-10-11T19:39:10.040Z,"188 km ENE of Saga, China",earthquake,10.43,1.807,0.080,48.0,reviewed,us,us


In [ ]:
df.shape

(1243, 22)

In [ ]:
print(df.columns)

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')


In [ ]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             1243 non-null   object 
 1   latitude         1243 non-null   float64
 2   longitude        1243 non-null   float64
 3   depth            1243 non-null   float64
 4   mag              1243 non-null   float64
 5   magType          1243 non-null   object 
 6   nst              369 non-null    float64
 7   gap              800 non-null    float64
 8   dmin             566 non-null    float64
 9   rms              1132 non-null   float64
 10  net              1243 non-null   object 
 11  id               1243 non-null   object 
 12  updated          1243 non-null   object 
 13  place            1243 non-null   object 
 14  type             1243 non-null   object 
 15  horizontalError  553 non-null    float64
 16  depthError       809 non-null    float64
 17  magError      

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy.stats import zscore
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
#1. Data Cleaning and Preprocessing
# Handle missing values
num_imputer = SimpleImputer(strategy='mean')
num_cols = df.select_dtypes(include=['float64']).columns
df[num_cols] = num_imputer.fit_transform(df[num_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])


In [ ]:
# Remove outliers using z-score
z_scores = np.abs(zscore(df[num_cols]))
df = df[(z_scores < 3).all(axis=1)]

In [ ]:
# Create new target variable based on magnitude thresholds
def classify_magnitude(mag):
    if mag > 5.5:
        return 'Fatal Earthquake'
    elif 4.5 <= mag <= 5.5:
        return 'Moderate Earthquake'
    elif 2.5 <= mag < 4.5:
        return 'Mild Earthquake'
    else:
        return 'Other'

df['Class'] = df['mag'].apply(classify_magnitude)


<ipython-input-27-edd33d710471>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class'] = df['mag'].apply(classify_magnitude)


In [ ]:
# Encode target variable
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])

<ipython-input-28-691f1efe1625>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class'] = le.fit_transform(df['Class'])


In [ ]:
# Encode categorical columns
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


<ipython-input-29-83106430a549>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
<ipython-input-29-83106430a549>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
<ipython-input-29-83106430a549>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [ ]:
# 2. Splitting the Data
X = df.drop(columns=['Class'])
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    matthews_corrcoef,
    confusion_matrix,
    cohen_kappa_score,
    classification_report,
)

# 1. Train the Random Forest Model
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# 2. Make Predictions
# Predict on the test data
rf_predictions = rf_model.predict(X_test)

# 3. Evaluate the Random Forest Model
print("Random Forest Model Evaluation:")

# Calculate basic metrics
accuracy = accuracy_score(y_test, rf_predictions)
precision = precision_score(y_test, rf_predictions, average='weighted')
recall = recall_score(y_test, rf_predictions, average='weighted')
f1 = f1_score(y_test, rf_predictions, average='weighted')

# Matthews Correlation Coefficient
mcc = matthews_corrcoef(y_test, rf_predictions)

# Cohen's Kappa Statistic
kappa = cohen_kappa_score(y_test, rf_predictions)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F-Measure (F1 Score): {f1:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"Cohen's Kappa Statistic: {kappa:.4f}")

# Display confusion matrix and classification report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, rf_predictions))

print("\nClassification Report:")
print(classification_report(y_test, rf_predictions))


Random Forest Model Evaluation:
Accuracy: 0.9953
Precision: 0.9953
Recall: 0.9953
F-Measure (F1 Score): 0.9948
Matthews Correlation Coefficient (MCC): 0.9908
Cohen's Kappa Statistic: 0.9907

Confusion Matrix:
[[  2   0   1]
 [  0 107   0]
 [  0   0 101]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       1.00      1.00      1.00       107
           2       0.99      1.00      1.00       101

    accuracy                           1.00       211
   macro avg       1.00      0.89      0.93       211
weighted avg       1.00      1.00      0.99       211



In [ ]:
from sklearn.dummy import DummyClassifier
# 3. Train ZeroR (Dummy Classifier)
# Initialize Dummy Classifier (ZeroR)
zeroR_model = DummyClassifier(strategy='most_frequent')  # ZeroR uses the most frequent class

# Train the ZeroR model
zeroR_model.fit(X_train, y_train)

# Make predictions
zeroR_predictions = zeroR_model.predict(X_test)

# 4. Evaluate ZeroR
print("ZeroR Evaluation:")

# Calculate metrics
accuracy = accuracy_score(y_test, zeroR_predictions)
precision = precision_score(y_test, zeroR_predictions, average='weighted')
recall = recall_score(y_test, zeroR_predictions, average='weighted')
f1 = f1_score(y_test, zeroR_predictions, average='weighted')
mcc = matthews_corrcoef(y_test, zeroR_predictions)
kappa = cohen_kappa_score(y_test, zeroR_predictions)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F-Measure (F1 Score): {f1:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"Cohen's Kappa Statistic: {kappa:.4f}")

# Display confusion matrix and classification report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, zeroR_predictions))

print("\nClassification Report:")
print(classification_report(y_test, zeroR_predictions))

ZeroR Evaluation:
Accuracy: 0.5071
Precision: 0.2572
Recall: 0.5071
F-Measure (F1 Score): 0.3413
Matthews Correlation Coefficient (MCC): 0.0000
Cohen's Kappa Statistic: 0.0000

Confusion Matrix:
[[  0   3   0]
 [  0 107   0]
 [  0 101   0]]

Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.51      1.00      0.67       107
           2       0.00      0.00      0.00       101

    accuracy                           0.51       211
   macro avg       0.17      0.33      0.22       211
weighted avg       0.26      0.51      0.34       211



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# 3. Train Logistic Regression
# Initialize Logistic Regression model
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)  # Increase max_iter for convergence

# Train the Logistic Regression model
log_reg_model.fit(X_train, y_train)

# Make predictions
log_reg_predictions = log_reg_model.predict(X_test)

# 4. Evaluate Logistic Regression
print("Logistic Regression Evaluation:")

# Calculate metrics
accuracy = accuracy_score(y_test, log_reg_predictions)
precision = precision_score(y_test, log_reg_predictions, average='weighted')
recall = recall_score(y_test, log_reg_predictions, average='weighted')
f1 = f1_score(y_test, log_reg_predictions, average='weighted')
mcc = matthews_corrcoef(y_test, log_reg_predictions)
kappa = cohen_kappa_score(y_test, log_reg_predictions)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F-Measure (F1 Score): {f1:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"Cohen's Kappa Statistic: {kappa:.4f}")

# Display confusion matrix and classification report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, log_reg_predictions))

print("\nClassification Report:")
print(classification_report(y_test, log_reg_predictions))

Logistic Regression Evaluation:
Accuracy: 0.8910
Precision: 0.8920
Recall: 0.8910
F-Measure (F1 Score): 0.8905
Matthews Correlation Coefficient (MCC): 0.7890
Cohen's Kappa Statistic: 0.7873

Confusion Matrix:
[[  1   0   2]
 [  0 101   6]
 [  2  13  86]]

Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.89      0.94      0.91       107
           2       0.91      0.85      0.88       101

    accuracy                           0.89       211
   macro avg       0.71      0.71      0.71       211
weighted avg       0.89      0.89      0.89       211



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, cohen_kappa_score, confusion_matrix, classification_report
)

# Assuming X_train, X_test, y_train, y_test are already defined
# Ensure X_train and X_test are NumPy arrays
X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test

# Scaling the Data (optional but helps LSTM converge faster)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape X_train and X_test to include a time dimension for LSTM
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])  # (samples, timesteps, features)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Convert Labels to Categorical for Multi-class Classification
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Build the LSTM Model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=y_train_cat.shape[1], activation='softmax'))

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train_cat, epochs=45, batch_size=32, validation_data=(X_test, y_test_cat))

# Make Predictions
y_pred = np.argmax(model.predict(X_test), axis=1)

# Evaluate the Model
print("LSTM Model Evaluation:")
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

# Display Metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F-Measure (F1 Score): {f1:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"Cohen's Kappa Statistic: {kappa:.4f}")

# Display Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.4547 - loss: 1.0768 - val_accuracy: 0.7867 - val_loss: 0.9434
Epoch 2/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7807 - loss: 0.9064 - val_accuracy: 0.7915 - val_loss: 0.7962
Epoch 3/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8249 - loss: 0.7518 - val_accuracy: 0.8104 - val_loss: 0.6731
Epoch 4/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8213 - loss: 0.6248 - val_accuracy: 0.8152 - val_loss: 0.5807
Epoch 5/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8369 - loss: 0.5161 - val_accuracy: 0.8246 - val_loss: 0.5241
Epoch 6/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8294 - loss: 0.4577 - val_accuracy: 0.8341 - val_loss: 0.4838
Epoch 7/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8580 - loss: 0.4199 - val_accuracy: 0.8483 - val_loss: 0.4545
Epoch 8/45
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8544 - loss: 0.3855 - val_accuracy: 0.8578 - val_los

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
LSTM Model Evaluation:
Accuracy: 0.9716
Precision: 0.9719
Recall: 0.9716
F-Measure (F1 Score): 0.9711
Matthews Correlation Coefficient (MCC): 0.9446
Cohen's Kappa Statistic: 0.9443

Confusion Matrix:
[[  2   0   1]
 [  0 106   1]
 [  0   4  97]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.96      0.99      0.98       107
           2       0.98      0.96      0.97       101

    accuracy                           0.97       211
   macro avg       0.98      0.87      0.92       211
weighted avg       0.97      0.97      0.97       211



In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
    matthews_corrcoef,
)

# Reshape X_train and X_test if they have more than 2 dimensions
if len(X_train.shape) > 2:
    X_train = X_train.reshape(X_train.shape[0], -1)
if len(X_test.shape) > 2:
    X_test = X_test.reshape(X_test.shape[0], -1)

# Flatten y_train and y_test to 1D if necessary
y_train = y_train.ravel()
y_test = y_test.ravel()

# Confirm the updated shapes
print("\nUpdated Shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Model Training
model = GaussianNB()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

# Display Results in Matrix Format
print("\nBayesNet Model Evaluation Metrics:")
print("+----------------------------------------------+")
print("| Metric                        | Value        |")
print("+----------------------------------------------+")
print(f"| Accuracy                      | {accuracy:.4f}   |")
print(f"| Precision (Weighted Average)  | {precision:.4f}   |")
print(f"| Recall (Weighted Average)     | {recall:.4f}   |")
print(f"| F1-Score (Weighted Average)   | {f1:.4f}   |")
print(f"| Matthews Correlation Coefficient (MCC) | {mcc:.4f}   |")
print(f"| Cohen's Kappa Statistic       | {kappa:.4f}   |")
print("+----------------------------------------------+")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Updated Shapes:
X_train shape: (844, 22)
X_test shape: (211, 22)
y_train shape: (844,)
y_test shape: (211,)

BayesNet Model Evaluation Metrics:
+----------------------------------------------+
| Metric                        | Value        |
+----------------------------------------------+
| Accuracy                      | 0.5545   |
| Precision (Weighted Average)  | 0.6507   |
| Recall (Weighted Average)     | 0.5545   |
| F1-Score (Weighted Average)   | 0.4461   |
| Matthews Correlation Coefficient (MCC) | 0.2080   |
| Cohen's Kappa Statistic       | 0.1119   |
+----------------------------------------------+

Confusion Matrix:
[[  2   0   1]
 [  0 105   2]
 [  1  90  10]]

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       0.54      0.98      0.70       107
           2       0.77      0.10      0.18       101

    accuracy                           0.55       211
   macro avg       

<ipython-input-35-d90d42523f3f>:21: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
<ipython-input-35-d90d42523f3f>:22: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel()


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
    matthews_corrcoef,
)
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Create a sample non-image dataset
# Increase the number of informative features to 5
X, y = make_classification(n_samples=1000, n_features=20, n_informative=5, n_classes=3, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode y_train and y_test for multi-class classification
num_classes = len(np.unique(y))
y_train = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

# Define the ANN model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dropout(0.5),
    Dense(64, activation='relu'),  # Second hidden layer
    Dense(num_classes, activation='softmax')  # Output layer with softmax activation for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test_categorical), epochs=10, batch_size=32)

# Predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

# Display Results in Matrix Format
print("\nANN Model Evaluation Metrics:")
print("+----------------------------------------------+")
print("| Metric                        | Value        |")
print("+----------------------------------------------+")
print(f"| Accuracy                      | {accuracy:.4f}   |")
print(f"| Precision (Weighted Average)  | {precision:.4f}   |")
print(f"| Recall (Weighted Average)     | {recall:.4f}   |")
print(f"| F1-Score (Weighted Average)   | {f1:.4f}   |")
print(f"| Matthews Correlation Coefficient (MCC) | {mcc:.4f}   |")
print(f"| Cohen's Kappa Statistic       | {kappa:.4f}   |")
print("+----------------------------------------------+")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3950 - loss: 1.1670 - val_accuracy: 0.5800 - val_loss: 0.9112
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6041 - loss: 0.9051 - val_accuracy: 0.7100 - val_loss: 0.7270
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6584 - loss: 0.7706 - val_accuracy: 0.7400 - val_loss: 0.6426
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6703 - loss: 0.7183 - val_accuracy: 0.8100 - val_loss: 0.5698
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7306 - loss: 0.6676 - val_accuracy: 0.7850 - val_loss: 0.5469
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7735 - loss: 0.5877 - val_accuracy: 0.8000 - val_loss: 0.5038
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7739 - loss: 0.5668 - val_accuracy: 0.8250 - val_loss: 0.4839
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8093 - loss: 0.4757 - val_accuracy: 0.8200 - val_loss: 0.4513
Ep

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, matthews_corrcoef
import numpy as np

# Reshape data for CNN (1D Convolution expects 3D input)
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the CNN model
cnn_model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(X_train_cnn, y_train, validation_data=(X_test_cnn, y_test_categorical), epochs=10, batch_size=32)

# Predictions for CNN
y_pred_probs_cnn = cnn_model.predict(X_test_cnn)
y_pred_cnn = np.argmax(y_pred_probs_cnn, axis=1)

# Metrics for CNN
accuracy_cnn = accuracy_score(y_test, y_pred_cnn)
precision_cnn = precision_score(y_test, y_pred_cnn, average='weighted')
recall_cnn = recall_score(y_test, y_pred_cnn, average='weighted')
f1_cnn = f1_score(y_test, y_pred_cnn, average='weighted')
mcc_cnn = matthews_corrcoef(y_test, y_pred_cnn)
kappa_cnn = cohen_kappa_score(y_test, y_pred_cnn)

print("\nCNN Model Evaluation Metrics:")
print(f"Accuracy: {accuracy_cnn:.4f}, Precision: {precision_cnn:.4f}, Recall: {recall_cnn:.4f}, F1-Score: {f1_cnn:.4f}, MCC: {mcc_cnn:.4f}, Kappa: {kappa_cnn:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_cnn))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_cnn))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.4396 - loss: 1.0706 - val_accuracy: 0.6150 - val_loss: 0.8754
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5962 - loss: 0.8781 - val_accuracy: 0.7050 - val_loss: 0.7531
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6376 - loss: 0.7925 - val_accuracy: 0.7000 - val_loss: 0.7093
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6861 - loss: 0.7118 - val_accuracy: 0.7100 - val_loss: 0.6631
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6991 - loss: 0.6764 - val_accuracy: 0.7650 - val_loss: 0.6014
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7273 - loss: 0.6692 - val_accuracy: 0.7650 - val_loss: 0.5830
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7069 - loss: 0.6303 - val_accuracy: 0.7150 - val_loss: 0.5972
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7559 - loss: 0.5789 - val_accuracy: 0.7750 - val_lo

In [ ]:
from tensorflow.keras.layers import SimpleRNN

# Reshape data for RNN (expects 3D input)
X_train_rnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_rnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the RNN model
rnn_model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(X_train_rnn.shape[1], 1)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the RNN model
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model.fit(X_train_rnn, y_train, validation_data=(X_test_rnn, y_test_categorical), epochs=10, batch_size=32)

# Predictions for RNN
y_pred_probs_rnn = rnn_model.predict(X_test_rnn)
y_pred_rnn = np.argmax(y_pred_probs_rnn, axis=1)

# Metrics for RNN
accuracy_rnn = accuracy_score(y_test, y_pred_rnn)
precision_rnn = precision_score(y_test, y_pred_rnn, average='weighted')
recall_rnn = recall_score(y_test, y_pred_rnn, average='weighted')
f1_rnn = f1_score(y_test, y_pred_rnn, average='weighted')
mcc_rnn = matthews_corrcoef(y_test, y_pred_rnn)
kappa_rnn = cohen_kappa_score(y_test, y_pred_rnn)

print("\nRNN Model Evaluation Metrics:")
print(f"Accuracy: {accuracy_rnn:.4f}, Precision: {precision_rnn:.4f}, Recall: {recall_rnn:.4f}, F1-Score: {f1_rnn:.4f}, MCC: {mcc_rnn:.4f}, Kappa: {kappa_rnn:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rnn))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rnn))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.3428 - loss: 1.1243 - val_accuracy: 0.4400 - val_loss: 1.0687
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3866 - loss: 1.0864 - val_accuracy: 0.4850 - val_loss: 1.0301
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4197 - loss: 1.0701 - val_accuracy: 0.5300 - val_loss: 1.0093
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4253 - loss: 1.0542 - val_accuracy: 0.5400 - val_loss: 0.9713
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4561 - loss: 1.0071 - val_accuracy: 0.5600 - val_loss: 0.9305
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5449 - loss: 0.9447 - val_accuracy: 0.6200 - val_loss: 0.8873
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5686 - loss: 0.8913 - val_accuracy: 0.6200 - val_loss: 0.8101
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6251 - loss: 0.8354 - val_accuracy: 0.6550 - val_loss: 0.7548
